In [0]:
!pip install python_speech_features

  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


# For MALE Gender

In [0]:
import os
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture
import python_speech_features as mfcc
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [0]:
import python_speech_features as mfcc # The following python code is a function to extract MFCC features from given audio.
def get_MFCC(sr,audio):
    features = mfcc.mfcc(audio, sr, 0.025, 0.01, 13, appendEnergy = False)
    features = preprocessing.scale(features)
    return features

In [83]:
#path to training data
source   = "pygender/train_data/youtube/male"
#path to save trained model
dest     = "pygender/"
files    = [os.path.join(source,f) for f in os.listdir(source) if
             f.endswith('.wav')]
features = np.asarray(());

print(files)

['pygender/train_data/youtube/male/male1.wav', 'pygender/train_data/youtube/male/male5.wav', 'pygender/train_data/youtube/male/male3.wav', 'pygender/train_data/youtube/male/male2.wav', 'pygender/train_data/youtube/male/male4.wav']


In [84]:
for f in files:
    
    sr,audio = read(f)
    vector   = get_MFCC(sr,audio)
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
        
print(f)
 
gmm = GaussianMixture(n_components = 8, max_iter = 200, covariance_type='diag',
        n_init = 3)
gmm.fit(features)
picklefile = f.split("/")[-2].split(".wav")[0]+".gmm"

print(picklefile)
 


pygender/train_data/youtube/male/male4.wav
male.gmm


In [87]:
# model saved as male.gmm
cPickle.dump(gmm,open(dest + picklefile,'wb'))
print('Modeling completed for gender:',picklefile.split(".gmm)")[0])


Modeling completed for gender: male.gmm


# For FEMALE Gender

In [0]:
import os
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture
import python_speech_features as mfcc
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [0]:
import python_speech_features as mfcc # The following python code is a function to extract MFCC features from given audio.
def get_MFCC(sr,audio):
    features = mfcc.mfcc(audio, sr, 0.025, 0.01, 13, appendEnergy = False)
    features = preprocessing.scale(features)
    return features

In [90]:
#path to training data
source   = "pygender/train_data/youtube/female"
#path to save trained model
dest     = "pygender/"
files    = [os.path.join(source,f) for f in os.listdir(source) if
             f.endswith('.wav')]
features = np.asarray(());

print(files)

['pygender/train_data/youtube/female/female4.wav', 'pygender/train_data/youtube/female/female3.wav', 'pygender/train_data/youtube/female/female5.wav', 'pygender/train_data/youtube/female/female2.wav', 'pygender/train_data/youtube/female/female1.wav']


In [91]:
for f in files:
    
    sr,audio = read(f)
    vector   = get_MFCC(sr,audio)
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
        
print(f)
 
gmm = GaussianMixture(n_components = 8, max_iter = 200, covariance_type='diag',
        n_init = 3)
gmm.fit(features)
picklefile = f.split("/")[-2].split(".wav")[0]+".gmm"

print(picklefile)

pygender/train_data/youtube/female/female1.wav
female.gmm


In [92]:
# model saved as male.gmm
cPickle.dump(gmm,open(dest + picklefile,'wb'))
print('Modeling completed for gender:',picklefile.split(".gmm)")[0])


Modeling completed for gender: female.gmm


# Gender Testing



In [0]:
import os
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture
import python_speech_features as mfcc
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [0]:
def get_MFCC(sr,audio):
    features = mfcc.mfcc(audio,sr, 0.025, 0.01, 13,appendEnergy = False)
    feat     = np.asarray(())
    for i in range(features.shape[0]):
        temp = features[i,:]
        if np.isnan(np.min(temp)):
            continue
        else:
            if feat.size == 0:
                feat = temp
            else:
                feat = np.vstack((feat, temp))
    features = feat;
    features = preprocessing.scale(features)
    return features

In [113]:
#path to test data
sourcepath = "pygender/"
#path to saved models
modelpath  = "pygender/"    
 
gmm_files = [os.path.join(modelpath,fname) for fname in
              os.listdir(modelpath) if fname.endswith('.gmm')]
print(gmm_files)

['pygender/male.gmm', 'pygender/female.gmm']


In [0]:


models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]

In [115]:
genders   = [fname.split("/")[-1].split(".gmm")[0] for fname
              in gmm_files]
files     = [os.path.join(sourcepath,f) for f in os.listdir(sourcepath)
              if f.endswith(".wav")] 
print(files)

['pygender/computer_two_hours.wav', 'pygender/computer_messages.wav', 'pygender/appeal_2_humanity.wav']


In [116]:
for f in files:
    print(f.split("\\")[-1])
    sr, audio  = read(f)
    features   = get_MFCC(sr,audio)
    scores     = None
    log_likelihood = np.zeros(len(models))
    for i in range(len(models)):
        gmm    = models[i]         #checking with each model one by one
        scores = np.array(gmm.score(features))
        log_likelihood[i] = scores.sum()
    winner = np.argmax(log_likelihood)
    print("\tdetected as - ", genders[winner],"\n\tscores:female ",log_likelihood[0],",male ", log_likelihood[1],"\n")


pygender/computer_two_hours.wav
	detected as -  female 
	scores:female  -19.545430239066043 ,male  -18.51087908537549 

pygender/computer_messages.wav
	detected as -  female 
	scores:female  -20.438728129480452 ,male  -19.6587751975247 

pygender/appeal_2_humanity.wav
	detected as -  male 
	scores:female  -18.104503936001667 ,male  -18.59609014074158 

